                                                    RDS CRUD OPERATION


'''<br>
@Author: Rahul<br> 
@Date: 2024-10-23<br>
@Last Modified by: Rahul <br>
@Last Modified time: 2024-10-23<br>
@Title: Python program to CRUD Operation using RDS instance database.<br>
'''

Establish Connection

In [3]:
import pymssql

In [14]:

server = 'rahulbldb-rj.cxsygi2uk94v.ap-south-1.rds.amazonaws.com'
user = 'admin'  
password = 'RaHuL637j'  
port = 1433  

connection = pymssql.connect(server=server, user=user, password=password, port=port,database='rdsrahul')
cursor = connection.cursor()
connection.autocommit(True)
print("Connection successful!")

Connection successful!


Create or Drop Database operation

In [15]:

try:
    cursor = connection.cursor()
    print("Connection successful!")
    
    new_database_name = input("Enter the name of the database: ")
    
    action = input("Do you want to create or drop the database? (create/drop): ").lower()
    
    if action == 'create':
        create_db_query = f"CREATE DATABASE {new_database_name}"
        cursor.execute(create_db_query)
        print(f"Database '{new_database_name}' created successfully!")
    
    elif action == 'drop':
        drop_db_query = f"DROP DATABASE IF EXISTS {new_database_name}"
        cursor.execute(drop_db_query)
        print(f"Database '{new_database_name}' dropped successfully!")
    
    else:
        print("Invalid option selected! Please enter either 'create' or 'drop'.")
    
except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Connection successful!
Database 'NewDatabseName' dropped successfully!


Create Table

In [8]:

try:
    table_name = 'Orders'
    
    create_table_query = f"""
    CREATE TABLE {table_name} (
        OrderID INT PRIMARY KEY IDENTITY(1,1),
        ProductID INT,
        OrderDate DATETIME DEFAULT GETDATE(),
        Quantity INT,
        FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
    );
    """
        
    cursor.execute(create_table_query)

    connection.commit()

    print(f"Table {table_name} created successfully!")

except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

OperationalError: (2714, b"There is already an object named 'Orders' in the database.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n")


Insert into table

In [11]:
try: 
    insert_query = f"""
    INSERT INTO {table_name} (ProductID, OrderDate, Quantity)
    VALUES (%d, %s, %d)
    """

    entity_data = [
        (1, '2024-10-22', 3),  
        (3, '2024-10-23', 1),  
        (1, '2024-10-24', 5)   
    ]

    cursor.executemany(insert_query, entity_data)

    connection.commit()

    print("Data inserted into Product table successfully!")

except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Data inserted into Product table successfully!


Update Table

In [59]:
try:
    update_query = """
    UPDATE Product
    SET ProductName = %s,
        Price = %d,
        StockQuantity = %d,
        SupplierID = %d
    WHERE ProductID = %d
    """
    
    product_update_data = ('Updated Smartphone', 32000, 180, 202, 2)

    cursor.execute(update_query, product_update_data)

    connection.commit()

    print("Data updated in Product table successfully!")

except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Data updated in Product table successfully!


Read Data

In [60]:
try:
    select_query = """
    SELECT *
    FROM Product
    """

    cursor.execute(select_query)

    rows = cursor.fetchall()

    if rows:
        print("Data retrieved from Product table:")
        for row in rows:
            print(f"ProductID: {row[0]}, ProductName: {row[1]}, Category: {row[2]}, Price: {row[3]}, StockQuantity: {row[4]}, SupplierID: {row[5]}, ManufactureDate: {row[6]}")
    else:
        print("No data found in Product table.")

except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Data retrieved from Product table:
ProductID: 1, ProductName: Laptop, Category: Electronics, Price: 75000.00, StockQuantity: 50, SupplierID: 201, ManufactureDate: 2023-05-10
ProductID: 2, ProductName: Updated Smartphone, Category: Electronics, Price: 32000.00, StockQuantity: 180, SupplierID: 202, ManufactureDate: 2024-02-15
ProductID: 3, ProductName: Refrigerator, Category: Appliances, Price: 45000.00, StockQuantity: 25, SupplierID: 203, ManufactureDate: 2023-12-01


Delete Record

In [61]:
try:
    delete_query = """
    DELETE FROM Product
    WHERE ProductID = %d
    """
    
    product_id_to_delete = 2  

    cursor.execute(delete_query, (product_id_to_delete,))

    connection.commit()

    print(f"Record with ProductID {product_id_to_delete} deleted successfully!")

except pymssql.OperationalError as e:
    print(f"OperationalError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Record with ProductID 2 deleted successfully!


Updating Stock and create Order

In [17]:
def update_stock_and_create_order(product_id, quantity):
    try:
        cursor = connection.cursor()
        
        cursor.execute("UPDATE Product SET StockQuantity = StockQuantity - %d WHERE ProductID = %d", (quantity, product_id))
        
        cursor.execute("INSERT INTO Orders (ProductID, OrderDate, Quantity) VALUES (%d, GETDATE(), %d)", (product_id, quantity))
        
        connection.commit()
        print("Transaction committed successfully!")
    
    except Exception as e:
        connection.rollback()
        print(f"An error occurred: {e}")

update_stock_and_create_order(1, 2)


Transaction committed successfully!
